In [1]:
import numpy as np
import pandas as pd
import scipy.sparse as sp
from tqdm import tqdm
from Base.DataIO import DataIO

#----Recommenders----
from SLIM.SLIM_BPR_Python import SLIM_BPR_Python
from SLIM.SlimElasticNet import SLIMElasticNetRecommender
from cf.item_cf3 import ItemBasedCollaborativeFiltering
from cf.user_cf2 import UserBasedCollaborativeFiltering
from MF.ALS import AlternatingLeastSquare
from cbf.cbf import ContentBasedFiltering
from SlimBPR.SlimBPRRec import SlimBPRRec
from SlimBPR.SlimBPR import SlimBPR

from Hybrid.hybridRec import HybridRecommender
#---------------

#----Model Load & Save----
from Data_manager.Dataset import Dataset
#-------------------------

from sklearn.model_selection import train_test_split

**Dataset loading with pandas**

The function read_csv from pandas provides a wonderful and fast interface to load tabular data like this. For better results and performance we provide the separator ::, the column names ["user_id", "item_id", "ratings", "timestamp"], and the types of each attribute in the dtype parameter.

In [ ]:
def load_data():
  return pd.read_csv("./data_train.csv")

In [ ]:
ratings=load_data()
d ={'user_id': ratings['row'],'item_id':ratings['col'],'ratings':ratings['data']}
ratings=pd.DataFrame(data=d)

In [ ]:
ratings.dtypes

In [ ]:
userList=list(d['user_id'])
itemList=list(d['item_id'])
ratingList=list(d['ratings'])

In [ ]:
URM = sp.coo_matrix((ratingList,(userList,itemList)))
URM = URM.tocsr()

In [ ]:
URM

In [2]:
def load_data_ICM():
  return pd.read_csv("./data_ICM_title_abstract.csv")

In [3]:
features=load_data_ICM()
d ={'item_id': features['row'],'feature_id':features['col'],'value':features['data']}
features=pd.DataFrame(data=d)
itemList=list(d['item_id'])

In [4]:
featureList=list(d['feature_id'])

In [5]:
valueList=list(d['value'])
ICM = sp.coo_matrix((valueList,(itemList,featureList)))
ICM = ICM.tocsr()

In [6]:
ICM

<25975x20000 sparse matrix of type '<class 'numpy.float64'>'
	with 490691 stored elements in Compressed Sparse Row format>

In [ ]:
num_users=URM.shape[0]

In [ ]:
num_items=URM.shape[1]

In [ ]:
num_users,num_items

**Dataset splitting into train,validation and test**

This is the last part before creating the recommender. However, this step is super important, as it is the base for the training, parameters optimization, and evaluation of the recommender(s).

In here we read the ratings (which we loaded and preprocessed before) and create the train, validation, and test User-Rating Matrices (URM). It's important that these are disjoint to avoid information leakage from the train into the validation/test set, in our case, we are safe to use the train_test_split function from scikit-learn as the dataset only contains one datapoint for every (user,item) pair. On another topic, we first create the test set and then we create the validation by splitting again the train set.

train_test_split takes an array (or several arrays) and divides it into train and test according to a given size (in our case testing_percentage and validation_percentage, which need to be a float between 0 and 1).

After we have our different splits, we create the sparse URMs by using the csr_matrix function from scipy.




In [ ]:
from Notebooks_utils.data_splitter import train_test_holdout

urm_train_validation, urm_test = train_test_holdout(URM, train_perc = 0.85)
urm_train, urm_validation = train_test_holdout(urm_train_validation, train_perc = 0.85)

In [ ]:
#Save Data
dataset = Dataset("prova", {"urm_train" : urm_train}, {"urm_validation" : urm_validation}, {"urm_test" : urm_test}) 
dataset.save_data("Salvataggi/")

In [7]:
#Load
loaded_dataset = Dataset(None, None, None, None)
loaded_dataset.load_data("Salvataggi/")


urm_train = (loaded_dataset.get_urm_train())["urm_train"]
urm_validation = (loaded_dataset.get_urm_validation())["urm_validation"]
urm_test = loaded_dataset.get_urm_test()["urm_test"]

In [8]:
urm_train_validation = urm_train + urm_validation

In [ ]:
from Base.Evaluation.Evaluator import EvaluatorHoldout

evaluator_validation = EvaluatorHoldout(urm_validation, cutoff_list=[10])
evaluator_test = EvaluatorHoldout(urm_test, cutoff_list=[10])

In [ ]:
from skopt.space import Real, Integer, Categorical

hyperparameters_range_dictionary = {}
hyperparameters_range_dictionary["knn"] = Integer(5, 1000)
hyperparameters_range_dictionary["shrink"] = Integer(0, 1000)
hyperparameters_range_dictionary["similarity"] = Categorical(["cosine"])

In [ ]:
from ParameterTuning.SearchBayesianSkopt import SearchBayesianSkopt

recommender_class = UserBasedCollaborativeFiltering

parameterSearch = SearchBayesianSkopt(recommender_class,
                                 evaluator_validation=evaluator_validation,
                                 evaluator_test=evaluator_test)

In [ ]:
from ParameterTuning.SearchAbstractClass import SearchInputRecommenderArgs
  
recommender_input_args = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS = [urm_train],     # For a CBF model simply put [URM_train, ICM_train]
    CONSTRUCTOR_KEYWORD_ARGS = {},
    FIT_POSITIONAL_ARGS = [],
    FIT_KEYWORD_ARGS = {}
)

In [ ]:
recommender_input_args_last_test = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS = [urm_train_validation],     # For a CBF model simply put [URM_train_validation, ICM_train]
    CONSTRUCTOR_KEYWORD_ARGS = {},
    FIT_POSITIONAL_ARGS = [],
    FIT_KEYWORD_ARGS = {}
)

In [ ]:
import os

output_folder_path = "result_experiments/"

# If directory does not exist, create
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)
    
n_cases = 50
n_random_starts = int(n_cases*0.3)
metric_to_optimize = "MAP"

In [ ]:
parameterSearch.search(recommender_input_args,
                       recommender_input_args_last_test = recommender_input_args_last_test,
                       parameter_search_space = hyperparameters_range_dictionary,
                       n_cases = n_cases,
                       n_random_starts = n_random_starts,
                       save_model = "last",
                       output_folder_path = output_folder_path,
                       output_file_name_root = recommender_class.RECOMMENDER_NAME,
                       metric_to_optimize = metric_to_optimize,
                      )

In [ ]:
data_loader = DataIO(folder_path = output_folder_path)
search_metadata = data_loader.load_data(recommender_class.RECOMMENDER_NAME + "_metadata.zip")
best_parameters = search_metadata["hyperparameters_best"]
best_parameters

**Evaluation Metrics**

In [9]:
def recall(recommendations: np.array, relevant_items: np.array) -> float:
    is_relevant = np.in1d(recommendations, relevant_items, assume_unique=True)
    
    recall_score = np.sum(is_relevant) / relevant_items.shape[0]
    
    return recall_score
    
    
def precision(recommendations: np.array, relevant_items: np.array) -> float:
    is_relevant = np.in1d(recommendations, relevant_items, assume_unique=True)
    
    precision_score = np.sum(is_relevant) / recommendations.shape[0]

    return precision_score

def mean_average_precision(recommendations: np.array, relevant_items: np.array) -> float:
    is_relevant = np.in1d(recommendations, relevant_items, assume_unique=True)
    
    precision_at_k = is_relevant * np.cumsum(is_relevant, dtype=np.float32) / (1 + np.arange(is_relevant.shape[0]))

    map_score = np.sum(precision_at_k) / np.min([relevant_items.shape[0], is_relevant.shape[0]])

    return map_score

**Evaluation Procedure**

The evaluation procedure returns the averaged accuracy scores (in terms of precision, recall and MAP) for all users (that have at least 1 rating in the test set). It also calculates the number of evaluated and skipped users. It receives a recommender instance, and the train and test URMs.

In [10]:
def evaluator(recommender: object, urm_train: sp.csr_matrix, urm_test: sp.csr_matrix):
    recommendation_length = 10
    accum_precision = 0
    accum_recall = 0
    accum_map = 0
    
    num_users = urm_train.shape[0]
    
    num_users_evaluated = 0
    num_users_skipped = 0
    
    for user_id in range(num_users):
        user_profile_start = urm_test.indptr[user_id]
        user_profile_end = urm_test.indptr[user_id+1]

        relevant_items = urm_test.indices[user_profile_start:user_profile_end]

        if relevant_items.size == 0:
            num_users_skipped += 1
            continue
            
#         recommendations = recommender.recommend(user_id_array=user_id,
#                                                cutoff=recommendation_length,
#                                                remove_seen_flag=True
#                                                )

        expected_ratings = recommender.get_expected_ratings(user_id)
        recommended_items = np.flip(np.argsort(expected_ratings), 0)

        unseen_items_mask = np.in1d(recommended_items,urm_train[user_id].indices,
                                        assume_unique=True, invert=True)

        recommendations = recommended_items[unseen_items_mask]


        accum_precision += precision(recommendations, relevant_items)
        accum_recall += recall(recommendations, relevant_items)
        accum_map += mean_average_precision(recommendations, relevant_items)

        num_users_evaluated += 1

    
    accum_precision /= max(num_users_evaluated, 1)
    accum_recall /= max(num_users_evaluated, 1)
    accum_map /=  max(num_users_evaluated, 1)
    
    return accum_precision, accum_recall, accum_map, num_users_evaluated, num_users_skipped

In [11]:
#Params for the recommenders
cbf_param = {
    "knn": 140,
    "shrink": 0
}

user_cf_param = {
    "knn": 249,
    "shrink": 853
}

item_cf_param = {
    "knn": 119,
    "shrink": 430
}

slim_bpr_param = {
    "learning_rate" : 0.05,
    "epochs": 10,
    "nnz" : 1,
    "knn": 200
}

als_param = {
    "n_factors": 300,
    "regularization": 0.15,
    "iterations": 30
}


In [ ]:
userCF = UserBasedCollaborativeFiltering(urm_train)

In [ ]:
userCF.fit(knn=user_cf_param["knn"], shrink=user_cf_param["shrink"],similarity='cosine')

In [ ]:
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped = evaluator(userCF, 
                                                                                            urm_train_validation, 
                                                                                            urm_test)
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped

In [ ]:
itemCF = ItemBasedCollaborativeFiltering(urm_train)

In [ ]:
itemCF.fit(knn=item_cf_param["knn"], shrink=item_cf_param["shrink"], similarity="cosine")

In [ ]:
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped = evaluator(itemCF, 
                                                                                            urm_train_validation, 
                                                                                            urm_test)
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped

In [ ]:
SlimBpr=SlimBPRRec(learning_rate=slim_bpr_param["learning_rate"], epochs=slim_bpr_param["epochs"], nnz=slim_bpr_param["nnz"], knn=slim_bpr_param["knn"])

In [ ]:
SlimBpr.fit(urm_train)

In [ ]:
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped = evaluator(SlimBpr, 
                                                                                            urm_train_validation, 
                                                                                            urm_test)
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped

In [ ]:
slim_elastic = SLIMElasticNetRecommender(urm_train)

In [ ]:
slim_elastic.fit()

In [ ]:
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped = evaluator(slim_elastic, 
                                                                                            urm_train_validation, 
                                                                                            urm_test)
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped

In [ ]:
cbf = ContentBasedFiltering(urm_train,ICM)

In [ ]:
cbf.fit(knn=cbf_param["knn"],shrink=cbf_param["shrink"],similarity='cosine')

In [ ]:
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped = evaluator(cbf, 
                                                                                            urm_train_validation, 
                                                                                            urm_test)
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped

In [ ]:
ALS = AlternatingLeastSquare(urm_train)

In [ ]:
ALS.fit(n_factors=als_param["n_factors"], regularization=als_param["regularization"],iterations=als_param["iterations"])

In [ ]:
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped = evaluator(ALS, 
                                                                                            urm_train_validation, 
                                                                                            urm_test)
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped

**Hybrid**

**Hybrid Recommender Tuning**

In [12]:
from Base.Evaluation.Evaluator import EvaluatorHoldout

evaluator_validation = EvaluatorHoldout(urm_validation, cutoff_list=[10])
evaluator_test = EvaluatorHoldout(urm_test, cutoff_list=[10])

In [13]:
from skopt.space import Real, Integer, Categorical

hyperparameters_range_dictionary = {}
hyperparameters_range_dictionary["w_user"] = Integer(0, 100)
hyperparameters_range_dictionary["w_item"] = Integer(0, 100)

In [14]:
from ParameterTuning.SearchBayesianSkopt import SearchBayesianSkopt

recommender_class = HybridRecommender

parameterSearch = SearchBayesianSkopt(recommender_class,
                                 evaluator_validation=evaluator_validation,
                                 evaluator_test=evaluator_test)

In [15]:
from ParameterTuning.SearchAbstractClass import SearchInputRecommenderArgs
  
recommender_input_args = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS = [urm_train, ICM],     # For a CBF model simply put [URM_train, ICM_train]
    CONSTRUCTOR_KEYWORD_ARGS = {},
    FIT_POSITIONAL_ARGS = [user_cf_param, item_cf_param, cbf_param, slim_bpr_param, als_param],
    FIT_KEYWORD_ARGS = {}
)

In [16]:
recommender_input_args_last_test = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS = [urm_train_validation, ICM],     # For a CBF model simply put [URM_train_validation, ICM_train]
    CONSTRUCTOR_KEYWORD_ARGS = {},
    FIT_POSITIONAL_ARGS = [user_cf_param, item_cf_param, cbf_param, slim_bpr_param, als_param],
    FIT_KEYWORD_ARGS = {}
)

In [17]:
import os

output_folder_path = "result_experiments/"

# If directory does not exist, create
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)
    
n_cases = 150
n_random_starts = int(n_cases*0.4)
metric_to_optimize = "MAP"

In [18]:
parameterSearch.search(recommender_input_args,
                       recommender_input_args_last_test = recommender_input_args_last_test,
                       parameter_search_space = hyperparameters_range_dictionary,
                       n_cases = n_cases,
                       n_random_starts = n_random_starts,
                       save_model = "last",
                       output_folder_path = output_folder_path,
                       output_file_name_root = recommender_class.RECOMMENDER_NAME,
                       metric_to_optimize = metric_to_optimize,
                      )

Iteration No: 1 started. Evaluating function at random point.
SearchBayesianSkopt: Testing config: {'w_user': 76, 'w_item': 3}
Fitting user cf...
Similarity column 25975 ( 100 % ), 3731.57 column/sec, elapsed time 0.12 min
Fitting item cf...
Similarity column 25975 ( 100 % ), 3895.38 column/sec, elapsed time 0.11 min
Fitting cbf...
Fitting slim bpr...
Fitting slim elastic...
Fitting ALS...
EvaluatorHoldout: Processed 4643 ( 100.00% ) in 2.60 sec. Users per second: 1786
SearchBayesianSkopt: New best config found. Config 0: {'w_user': 76, 'w_item': 3} - results: ROC_AUC: 0.1071655, PRECISION: 0.0207409, PRECISION_RECALL_MIN_DEN: 0.0871684, RECALL: 0.0854813, MAP: 0.0390993, MRR: 0.0759772, NDCG: 0.0586599, F1: 0.0333821, HIT_RATE: 0.2074090, ARHR: 0.0828714, NOVELTY: 0.0047813, AVERAGE_POPULARITY: 0.1276741, DIVERSITY_MEAN_INTER_LIST: 0.9748846, DIVERSITY_HERFINDAHL: 0.9974675, COVERAGE_ITEM: 0.3824832, COVERAGE_ITEM_CORRECT: 0.0184408, COVERAGE_USER: 0.5842456, COVERAGE_USER_CORRECT: 0.

EvaluatorHoldout: Processed 5007 ( 100.00% ) in 2.81 sec. Users per second: 1781
SearchBayesianSkopt: Best config evaluated with evaluator_test. Config: {'w_user': 2, 'w_item': 45} - results:
CUTOFF: 10 - ROC_AUC: 0.1144559, PRECISION: 0.0227482, PRECISION_RECALL_MIN_DEN: 0.0873378, RECALL: 0.0854196, MAP: 0.0388410, MRR: 0.0828509, NDCG: 0.0597634, F1: 0.0359282, HIT_RATE: 0.2274815, ARHR: 0.0918069, NOVELTY: 0.0048474, AVERAGE_POPULARITY: 0.1153955, DIVERSITY_MEAN_INTER_LIST: 0.9794373, DIVERSITY_HERFINDAHL: 0.9979242, COVERAGE_ITEM: 0.4132820, COVERAGE_ITEM_CORRECT: 0.0232916, COVERAGE_USER: 0.6300491, COVERAGE_USER_CORRECT: 0.1151378, DIVERSITY_GINI: 0.1270945, SHANNON_ENTROPY: 11.1045349, 


Iteration No: 5 ended. Evaluation done at random point.
Time taken: 21.0386
Function value obtained: -0.0395
Current minimum: -0.0395
Iteration No: 6 started. Evaluating function at random point.
SearchBayesianSkopt: Testing config: {'w_user': 18, 'w_item': 15}
Fitting user cf...
Similarity co


Iteration No: 11 ended. Evaluation done at random point.
Time taken: 18.7019
Function value obtained: -0.0396
Current minimum: -0.0397
Iteration No: 12 started. Evaluating function at random point.
SearchBayesianSkopt: Testing config: {'w_user': 62, 'w_item': 96}
Fitting user cf...
Similarity column 25975 ( 100 % ), 3926.02 column/sec, elapsed time 0.11 min
Fitting item cf...
Similarity column 25975 ( 100 % ), 4211.71 column/sec, elapsed time 0.10 min
Fitting cbf...
Fitting slim bpr...
Fitting slim elastic...
Fitting ALS...
EvaluatorHoldout: Processed 4643 ( 100.00% ) in 2.57 sec. Users per second: 1806
SearchBayesianSkopt: Config 11 is suboptimal. Config: {'w_user': 62, 'w_item': 96} - results: ROC_AUC: 0.1083744, PRECISION: 0.0205686, PRECISION_RECALL_MIN_DEN: 0.0866297, RECALL: 0.0850029, MAP: 0.0394559, MRR: 0.0770028, NDCG: 0.0589085, F1: 0.0331224, HIT_RATE: 0.2056860, ARHR: 0.0839947, NOVELTY: 0.0048160, AVERAGE_POPULARITY: 0.1196797, DIVERSITY_MEAN_INTER_LIST: 0.9778284, DIVER

SearchBayesianSkopt: Testing config: {'w_user': 9, 'w_item': 87}
Fitting user cf...
Similarity column 25975 ( 100 % ), 3771.90 column/sec, elapsed time 0.11 min
Fitting item cf...
Similarity column 25975 ( 100 % ), 4167.39 column/sec, elapsed time 0.10 min
Fitting cbf...
Fitting slim bpr...
Fitting slim elastic...
Fitting ALS...
EvaluatorHoldout: Processed 4643 ( 100.00% ) in 2.57 sec. Users per second: 1808
SearchBayesianSkopt: Config 17 is suboptimal. Config: {'w_user': 9, 'w_item': 87} - results: ROC_AUC: 0.1082443, PRECISION: 0.0205040, PRECISION_RECALL_MIN_DEN: 0.0865054, RECALL: 0.0848721, MAP: 0.0396440, MRR: 0.0770833, NDCG: 0.0589614, F1: 0.0330287, HIT_RATE: 0.2050398, ARHR: 0.0839008, NOVELTY: 0.0048251, AVERAGE_POPULARITY: 0.1177963, DIVERSITY_MEAN_INTER_LIST: 0.9786043, DIVERSITY_HERFINDAHL: 0.9978394, COVERAGE_ITEM: 0.3926853, COVERAGE_ITEM_CORRECT: 0.0188643, COVERAGE_USER: 0.5842456, COVERAGE_USER_CORRECT: 0.1001636, DIVERSITY_GINI: 0.1197461, SHANNON_ENTROPY: 11.018531


Iteration No: 24 ended. Evaluation done at random point.
Time taken: 18.6646
Function value obtained: -0.0395
Current minimum: -0.0397
Iteration No: 25 started. Evaluating function at random point.
SearchBayesianSkopt: Testing config: {'w_user': 44, 'w_item': 34}
Fitting user cf...
Similarity column 25975 ( 100 % ), 3741.03 column/sec, elapsed time 0.12 min
Fitting item cf...
Similarity column 25975 ( 100 % ), 4160.98 column/sec, elapsed time 0.10 min
Fitting cbf...
Fitting slim bpr...
Fitting slim elastic...
Fitting ALS...
EvaluatorHoldout: Processed 4643 ( 100.00% ) in 2.57 sec. Users per second: 1807
SearchBayesianSkopt: Config 24 is suboptimal. Config: {'w_user': 44, 'w_item': 34} - results: ROC_AUC: 0.1080948, PRECISION: 0.0206117, PRECISION_RECALL_MIN_DEN: 0.0870528, RECALL: 0.0854202, MAP: 0.0396175, MRR: 0.0772166, NDCG: 0.0591187, F1: 0.0332099, HIT_RATE: 0.2061167, ARHR: 0.0841842, NOVELTY: 0.0048116, AVERAGE_POPULARITY: 0.1207191, DIVERSITY_MEAN_INTER_LIST: 0.9774450, DIVER

...

C:\Users\davel\anaconda3\envs\RecSysFramework\lib\site-packages\skopt\optimizer\optimizer.py:399: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


SearchBayesianSkopt: Testing config: {'w_user': 1, 'w_item': 100}
Fitting user cf...
Similarity column 25975 ( 100 % ), 3649.57 column/sec, elapsed time 0.12 min
Fitting item cf...
Similarity column 25975 ( 100 % ), 4006.18 column/sec, elapsed time 0.11 min
Fitting cbf...
Fitting slim bpr...
Fitting slim elastic...
Fitting ALS...
EvaluatorHoldout: Processed 4643 ( 100.00% ) in 2.64 sec. Users per second: 1761
SearchBayesianSkopt: Config 148 is suboptimal. Config: {'w_user': 1, 'w_item': 100} - results: ROC_AUC: 0.1072693, PRECISION: 0.0204178, PRECISION_RECALL_MIN_DEN: 0.0858539, RECALL: 0.0841913, MAP: 0.0392406, MRR: 0.0764440, NDCG: 0.0584417, F1: 0.0328653, HIT_RATE: 0.2041783, ARHR: 0.0832882, NOVELTY: 0.0048360, AVERAGE_POPULARITY: 0.1170430, DIVERSITY_MEAN_INTER_LIST: 0.9788130, DIVERSITY_HERFINDAHL: 0.9978602, COVERAGE_ITEM: 0.3969971, COVERAGE_ITEM_CORRECT: 0.0187488, COVERAGE_USER: 0.5842456, COVERAGE_USER_CORRECT: 0.0991569, DIVERSITY_GINI: 0.1222709, SHANNON_ENTROPY: 11.042

C:\Users\davel\anaconda3\envs\RecSysFramework\lib\site-packages\skopt\optimizer\optimizer.py:399: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


SearchBayesianSkopt: Testing config: {'w_user': 1, 'w_item': 100}
Fitting user cf...
Similarity column 25975 ( 100 % ), 3822.62 column/sec, elapsed time 0.11 min
Fitting item cf...
Similarity column 25975 ( 100 % ), 4082.91 column/sec, elapsed time 0.11 min
Fitting cbf...
Fitting slim bpr...
Fitting slim elastic...
Fitting ALS...
EvaluatorHoldout: Processed 4643 ( 100.00% ) in 2.58 sec. Users per second: 1797
SearchBayesianSkopt: Config 149 is suboptimal. Config: {'w_user': 1, 'w_item': 100} - results: ROC_AUC: 0.1072693, PRECISION: 0.0204178, PRECISION_RECALL_MIN_DEN: 0.0858539, RECALL: 0.0841913, MAP: 0.0392406, MRR: 0.0764440, NDCG: 0.0584417, F1: 0.0328653, HIT_RATE: 0.2041783, ARHR: 0.0832882, NOVELTY: 0.0048360, AVERAGE_POPULARITY: 0.1170430, DIVERSITY_MEAN_INTER_LIST: 0.9788130, DIVERSITY_HERFINDAHL: 0.9978602, COVERAGE_ITEM: 0.3969971, COVERAGE_ITEM_CORRECT: 0.0187488, COVERAGE_USER: 0.5842456, COVERAGE_USER_CORRECT: 0.0991569, DIVERSITY_GINI: 0.1222709, SHANNON_ENTROPY: 11.042

TypeError: get_expected_ratings() missing 1 required positional argument: 'user_id'

In [ ]:
data_loader = DataIO(folder_path = output_folder_path)
search_metadata = data_loader.load_data(recommender_class.RECOMMENDER_NAME + "_metadata.zip")
best_parameters = search_metadata["hyperparameters_best"]
best_parameters

In [19]:
w = {
    "user_cf": 5,
    "item_cf": 30,
    "cbf": 0,
    "icm_svd": 0,
    "als": 0,
    "slim": 0,
    "elastic": 0
}

In [23]:
recommender = HybridRecommender(urm_train_validation, ICM)

In [24]:
recommender.fit(user_cf_param=user_cf_param,item_cf_param=item_cf_param,cbf_param=cbf_param,
                slim_param=slim_bpr_param,als_param=als_param, w_user=w["user_cf"], w_item=w["item_cf"])

Fitting user cf...
Similarity column 25975 ( 100 % ), 2454.32 column/sec, elapsed time 0.18 min
Fitting item cf...
Similarity column 25975 ( 100 % ), 3846.03 column/sec, elapsed time 0.11 min
Fitting cbf...
Fitting slim bpr...
Fitting slim elastic...
Fitting ALS...


In [25]:
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped = evaluator(recommender, 
                                                                                            urm_train_validation, 
                                                                                            urm_test)
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped

(0.00013115158761398967, 1.0, 0.057309183461753455, 5007, 2940)

**Submission to competition**

This step serves as a similar step that you will perform when preparing a submission to the competition. Specially after you have chosen and trained your recommender.

For this step the best suggestion is to select the most-performing configuration obtained in the hyperparameter tuning step and to train the recommender using both the train and validation set. Remember that in the competition you do not have access to the test set.

Another consideration is that, due to easier and faster calculations, we replaced the user/item identifiers with new ones in the preprocessing step. For the competition, you are required to generate recommendations using the dataset's original identifiers. Due to this, this step also reverts back the newer identifiers with the ones originally found in the dataset.

Last, this step creates a function that writes the recommendations for each user in the same file in a tabular format following this format:

csv
<user_id>,<item_id_1> <item_id_2> <item_id_3> <item_id_4> <item_id_5> <item_id_6> <item_id_7> <item_id_8> <item_id_9> <item_id_10>
Always verify the competitions' submission file model as it might vary from the one we presented here.

In [26]:
def load_goodguys():
  return pd.read_csv("./data_target_users_test.csv")
goodguys=load_goodguys()

In [27]:
goodguys

,user_id
0,0
1,1
2,2
3,3
4,4
5,5
6,6
7,7
8,8
9,9


In [28]:
users_to_recommend = np.random.choice(goodguys.user_id,size=goodguys.size, replace=False)
users_to_recommend

array([1216, 6502, 5022, ..., 1776, 7171, 5986], dtype=int64)

In [29]:
def prepare_submission(users_to_recommend: np.array, urm_train: sp.csr_matrix, recommender: object):
    
    recommendation_length = 10
    submission = []
    
    for user_id in users_to_recommend :

        recommendations = recommender.recommend2(user_id, urm_train, recommendation_length)

        
        submission.append((user_id, [item_id for item_id in recommendations]))
   
    return submission

In [30]:
submission = prepare_submission(users_to_recommend, urm_train_validation, recommender)

In [31]:
submission

[(1216, [24355, 20308, 6329, 19709, 19874, 14684, 9343, 16860, 1811, 10396]),
 (6502, [25635, 8309, 23391, 25138, 10792, 13136, 17209, 23953, 18208, 2635]),
 (5022, [2426, 23154, 25407, 9555, 10594, 18392, 8316, 15691, 10792, 22848]),
 (7585, [2062, 1276, 3498, 20348, 14625, 7445, 22364, 20641, 17219, 23622]),
 (6403, [9339, 15830, 8544, 20108, 1863, 12273, 24410, 6521, 9243, 19910]),
 (5891, [19340, 8395, 24932, 15645, 2322, 5908, 22949, 6981, 5800, 15795]),
 (2061, [24355, 16562, 24481, 14684, 15778, 4257, 17668, 19874, 13073, 3431]),
 (7484, [25407, 5952, 25596, 2426, 12050, 8860, 2492, 23154, 8097, 18452]),
 (2693, [25407, 8990, 16772, 15691, 23141, 10834, 4878, 8097, 15196, 1240]),
 (4772, [3884, 10408, 22898, 41, 10780, 8669, 8654, 8655, 8656, 8673]),
 (2486, [23842, 1763, 17168, 8654, 8659, 8655, 8656, 8657, 8658, 8653]),
 (1027, [16394, 22010, 25407, 22731, 4257, 11713, 12165, 19815, 3004, 19948]),
 (787, [19480, 12409, 19709, 16630, 20146, 7494, 20095, 17723, 2665, 14895]),
 (

In [32]:
import os
from datetime import datetime

csv_fname = './submission'
csv_fname += datetime.now().strftime('%b%d_%H-%M-%S') + '.csv'

def write_submission(submissions):
    with open(csv_fname, "w") as f:
        f.write(f"user_id,item_list\n")
        for user_id, items in submissions:
            f.write(f"{user_id},{' '.join([str(item) for item in items])}\n")


In [33]:
write_submission(submission)